In [60]:
:dep caw = { path = "/Users/s/src/caw/caw", features = ["live", "widgets"] }

In [61]:
use caw::prelude::*;
let out: LiveStereoOut = 
    live_stereo_viz_udp(Default::default())
        .with_volume(knob("volume").build());

In [62]:
let lpf_cutoff: SV<f32> = sv(knob("low pass cutofff").build());

In [63]:
let lpf_resonance: SV<f32> = sv(knob("low pass res").build());

In [100]:
let kick_lpf_cutoff: SV<f32> = sv(knob("kick lpf").build());

In [107]:
let kick_lpf_q: SV<f32> = sv(knob("kick q").build());

In [81]:
let pwm: SV<f32> = sv(knob("pwm").build());

In [127]:
let clock_hz: SV<f32> = sv_default();
let clock: SV<bool> = sv_default();

In [136]:
clock_hz.set(8.);


thread 'main' panicked at src/main.rs:77:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7: _main

thread 'main' panicked at src/main.rs:77:22:
called `Result::unwrap()` on an `Err` value: Connection refused (os error 61)

Stack backtrace:
   0: std::backtrace::Backtrace::create
   1: anyhow::error::<impl core::convert::From<E> for anyhow::Error>::from
   2: caw_midi_udp_client::MidiUdpClient::send
   3: caw_midi_udp_widgets_app::main
   4: std::sys::backtrace::__rust_begin_short_backtrace
   5: std::rt::lang_start::{{closure}}
   6: std::rt::lang_start_internal
   7: _main

thread 'main' panicked at src/mai

In [129]:
let snare_clock: SV<bool> = sv(clock.clone().divide_with_offset(2, 1));
let kick_clock: SV<bool> = sv(clock.clone().divide_with_offset(2, 0));

In [130]:
clock.set(periodic_trig_hz(clock_hz.clone()).build());

In [131]:
let chord_progression = {
};

In [135]:
out.set_channel(|channel| {
    let filter_env = adsr(kick_clock.clone()).r(0.2).build().shared();
    
    let freq = match channel {
        Channel::Left => (20. +200. * filter_env.clone()).boxed(),
        Channel::Right => (20. + 200. * filter_env.clone()).boxed(),
    };
    
    let kick_sine = (oscillator(Sine, freq)
        .pulse_width_01(pwm.clone())
        .reset_offset_01(channel.circle_phase_offset_01())
        .build() * filter_env.clone());
    let kick_noise = noise::white()
        .filter(low_pass::default(filter_env.clone() * 10000. * kick_lpf_cutoff.clone()).q(2. * kick_lpf_q.clone()));
   


    let snare_filter_env = adsr(snare_clock.clone()).r(0.1).build().shared();
        
    let snare_noise = 
        noise::white()
        .filter(low_pass::default(snare_filter_env.clone() * 15000. * kick_lpf_cutoff.clone()).q(2. * kick_lpf_q.clone()));

     kick_sine  + kick_noise + snare_noise
    
        //.filter(low_pass::default(lpf_cutoff.clone() * filter_env * 10_000.).q(lpf_resonance.clone()))
        //.filter(compressor().scale(4.).threshold(0.5).ratio(0.1))
        //.filter(chorus())
       // .filter(reverb())
        //.filter(high_pass_butterworth(1.))
});